In [1]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00


In [2]:
import openai
import pandas as pd
from google.colab import userdata

openai.api_key = userdata.get('OPENAI_API_KEY')

In [3]:
#Load the Responses
df = pd.read_csv("/content/test_results_llama_3_8B_instruct_bge-base-en-v1.5__768_1024_50_20_5.csv")
df.head()

,Unnamed: 0,question,ground_truth,category,llm_only_response,rag_response,embed_model,embedding_dimension,chunk_size,chunk_overlap,similarity_top_k,rerank_top_n
0,0,What is required for an F-1 student to take a ...,Authorization from a DSO and documentation fro...,F-1,To take a reduced course load due to medical r...,"According to the provided policy manual, an F-...",BAAI/bge-base-en-v1.5,768,1024,50,20,5
1,1,Can F-1 students enroll in two SEVP-certified ...,"Yes, if the combined enrollment amounts to a f...",F-1,According to the Student and Exchange Visitor ...,"Yes, F-1 students can enroll in multiple SEVP-...",BAAI/bge-base-en-v1.5,768,1024,50,20,5
2,2,Under what condition can a DSO authorize a red...,When the student needs fewer courses to comple...,F-1,A Degree-Seeking Organization (DSO) can author...,"Based on the provided context information, a D...",BAAI/bge-base-en-v1.5,768,1024,50,20,5
3,3,Are F-1 visa holders allowed to study at publi...,"Yes, but only at SEVP-certified public high sc...",F-1,"F-1 visa holders, also known as international ...","Based on the provided policy manual, it appear...",BAAI/bge-base-en-v1.5,768,1024,50,20,5
4,4,What must F-1 students provide to attend a pub...,"Proof of reimbursement for the full, unsubsidi...",F-1,F-1 international students who wish to attend ...,"According to the provided policy manual, F-1 s...",BAAI/bge-base-en-v1.5,768,1024,50,20,5


In [4]:
from openai import OpenAI

client = OpenAI(api_key = openai.api_key)

In [5]:
#Define Function for Scoring the Responses
import re

def evaluate_response(question, ground_truth, llm_only_response, rag_response):
    prompt = f"""
    Question: {question}
    Ground Truth: {ground_truth}
    LLM Only Response: {llm_only_response}
    RAG Response: {rag_response}

    Compare the LLM only response and the RAG response to the ground truth respectively on the following criteria:
    - Relevance: Does the response directly address the query?
    - Accuracy: Is the information provided correct?
    - Completeness: Does the response cover all necessary aspects of the query?
    - Clarity: Is the response easy to understand?
    - Conciseness: Is the response free from unnecessary information?

    Thereafter, provide an overall score out of 10 for LLM Only Response and for the RAG Response respectively. Do not change the format of the output.

    ### Summary
    - LLM Only Response:

    - RAG Response:

    ### LLM Only Overall Score:
    ### RAG Overall Score:
    """

    response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "You are a helpful assistant that evaluates responses"},
    {"role": "user", "content": prompt}
  ]
)

    s = response.choices[0].message.content
    print(s)

    llm_only_score = re.search(r'LLM Only Overall Score: (\d+)', s).group(1).strip()
    rag_score = re.search(r'RAG Overall Score: (\d+)', s).group(1).strip()

    return int(llm_only_score), int(rag_score)


In [6]:
from tqdm import tqdm

for i in tqdm(range(len(df))):
  df.loc[i, 'llm_only_score'], df.loc[i, 'rag_score'] = evaluate_response(df.loc[i, 'question'],
                                                                 df.loc[i, 'ground_truth'],
                                                                 df.loc[i, 'llm_only_response'],
                                                                 df.loc[i, 'rag_response'])

  1%|          | 1/100 [00:04<06:58,  4.22s/it]

### Summary
- LLM Only Response:
  - Relevance: Yes, the response directly addresses the query about taking a reduced course load due to medical reasons for an F-1 student.
  - Accuracy: The information provided is generally correct, including the requirement for documentation from a licensed medical professional and obtaining approval from a DSO.
  - Completeness: The response is quite comprehensive, detailing several steps involved in the process, which helps cover necessary aspects of the query.
  - Clarity: The information is presented in a clear, step-by-step manner, making it easy to understand.
  - Conciseness: While informative, the response could be slightly more concise as it includes some repetitive or redundant details.

- RAG Response:
  - Relevance: Yes, the RAG response also directly addresses the query, focusing on the requirements for reduced course load due to medical reasons.
  - Accuracy: The information provided is accurate and includes specific requirements for do

  2%|▏         | 2/100 [00:08<06:39,  4.08s/it]

### Summary
- LLM Only Response:
  - Relevance: The response directly addresses the query but incorrectly states that F-1 students are not allowed to enroll in multiple SEVP-certified schools, which is a crucial aspect of the question.
  - Accuracy: The information provided is mostly inaccurate, as it contradicts the ground truth that allows concurrent enrollment under specific conditions.
  - Completeness: While the LLM response provides additional context about enrollment regulations, it ultimately fails to present the essential point of the question correctly.
  - Clarity: The response is clear in its language and structure but is misleading due to the incorrect information.
  - Conciseness: The response includes unnecessary detail about exceptions that are not applicable to the ground truth statement.

- RAG Response:
  - Relevance: The response is directly relevant to the query and addresses the question of concurrent enrollment.
  - Accuracy: The information provided is accurate 

  3%|▎         | 3/100 [00:12<06:37,  4.09s/it]

...

### Summary
- LLM Only Response:
  - Relevance: The response directly addresses the query by listing documents required to prove continuous lawful status in the U.S. for an adjustment of status application.
  - Accuracy: The information provided is correct, mentioning key documents such as Form I-94 and others relevant for proving lawful status.
  - Completeness: The response is comprehensive, covering various types of documentation and mentioning additional considerations based on individual situations.
  - Clarity: The response is well-structured and easy to understand, with clearly delineated sections for each type of document.
  - Conciseness: The response could be considered slightly lengthy, but it is mostly free from unnecessary information, given the complexity of the topic.

- RAG Response:
  - Relevance: The response addresses the query by discussing the burden of proof and the types of evidence required to demonstrate continuous lawful status, albeit less directly.
  - Accur

 99%|█████████▉| 99/100 [09:28<00:05,  5.64s/it]

### Summary
- LLM Only Response:
  - **Relevance:** The response directly addresses the query by defining Forms I-864 and I-864EZ and their role in the Green Card application process. 
  - **Accuracy:** The information is correct regarding the functions and requirements of both forms.
  - **Completeness:** The response provides thorough details about each form, including sponsorship obligations, income requirements, and specific use cases for each form. 
  - **Clarity:** The response is well-structured and easy to understand, with clear explanations of each form’s purpose and requirements.
  - **Conciseness:** The response is relatively concise while still being informative, avoiding unnecessary information.

- RAG Response:
  - **Relevance:** The response addresses the query but is slightly less focused than the LLM response, as it repeats definitions without adding significant new information.
  - **Accuracy:** The information given about Forms I-864 and I-864EZ is correct, but it la

100%|██████████| 100/100 [09:35<00:00,  5.75s/it]

### Summary
- LLM Only Response:
  - **Relevance**: High - The response directly addresses the process for family members of a U.S. citizen's immediate relative to apply for a Green Card.
  - **Accuracy**: High - The steps involved in the Green Card application process are correctly outlined and relevant to immediate relatives.
  - **Completeness**: High - The response thoroughly covers the application process, including eligibility, forms to file, and the steps involved.
  - **Clarity**: High - The response is well-organized and easy to follow, using clearly labeled steps.
  - **Conciseness**: Moderate - While detailed, the response is longer than necessary in some areas, potentially introducing minor redundancy.

- RAG Response:
  - **Relevance**: Low - The response focuses on a process specific to T nonimmigrants, which is not directly relevant to the query about immediate relatives of U.S. citizens applying for a Green Card.
  - **Accuracy**: Moderate - While the information regard

In [7]:
df

,Unnamed: 0,question,ground_truth,category,llm_only_response,rag_response,embed_model,embedding_dimension,chunk_size,chunk_overlap,similarity_top_k,rerank_top_n,llm_only_score,rag_score
0,0,What is required for an F-1 student to take a ...,Authorization from a DSO and documentation fro...,F-1,To take a reduced course load due to medical r...,"According to the provided policy manual, an F-...",BAAI/bge-base-en-v1.5,768,1024,50,20,5,8.0,9.0
1,1,Can F-1 students enroll in two SEVP-certified ...,"Yes, if the combined enrollment amounts to a f...",F-1,According to the Student and Exchange Visitor ...,"Yes, F-1 students can enroll in multiple SEVP-...",BAAI/bge-base-en-v1.5,768,1024,50,20,5,4.0,8.0
2,2,Under what condition can a DSO authorize a red...,When the student needs fewer courses to comple...,F-1,A Degree-Seeking Organization (DSO) can author...,"Based on the provided context information, a D...",BAAI/bge-base-en-v1.5,768,1024,50,20,5,6.0,9.0
3,3,Are F-1 visa holders allowed to study at publi...,"Yes, but only at SEVP-certified public high sc...",F-1,"F-1 visa holders, also known as international ...","Based on the provided policy manual, it appear...",BAAI/bge-base-en-v1.5,768,1024,50,20,5,3.0,8.0
4,4,What must F-1 students provide to attend a pub...,"Proof of reimbursement for the full, unsubsidi...",F-1,F-1 international students who wish to attend ...,"According to the provided policy manual, F-1 s...",BAAI/bge-base-en-v1.5,768,1024,50,20,5,5.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,Describe the eligibility requirements for obta...,"To be eligible for a Green Card, you must file...",Green Card,The coveted Green Card!\n\nIn the United State...,"Based on the provided context information, I'l...",BAAI/bge-base-en-v1.5,768,1024,50,20,5,8.0,5.0
96,96,How can one apply for employment authorization...,By filing Form I-765 for employment authorizat...,Green Card,"When you have a pending Form I-485, Applicatio...",When an alien is returned to the custody of th...,BAAI/bge-base-en-v1.5,768,1024,50,20,5,9.0,2.0
97,97,Describe the documents required to prove conti...,The required documents include a copy of Form ...,Green Card,To prove continuous lawful status in the Unite...,"In matters involving immigration benefits, the...",BAAI/bge-base-en-v1.5,768,1024,50,20,5,8.0,5.0
98,98,Define Form I-864 and Form I-864EZ in the cont...,Form I-864 and Form I-864EZ are affidavits of ...,Green Card,In the context of applying for a Green Card (L...,"Based on the provided context information, For...",BAAI/bge-base-en-v1.5,768,1024,50,20,5,9.0,7.0


In [8]:
import numpy as np

np.unique(df['llm_only_score'])

array([ 3.,  4.,  5.,  6.,  7.,  8.,  9., 43.])

In [9]:
np.unique(df['rag_score'])

array([ 2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 43.])

In [10]:
import numpy as np

df['llm_only_score'] = np.where(df['llm_only_score'] > 10, 10*df['llm_only_score']/50, df['llm_only_score'])
df['rag_score'] =  np.where(df['rag_score'] > 10, 10*df['rag_score']/50, df['rag_score'])

In [11]:
print(f"Avg. LLM Only Score: {df['llm_only_score'].mean()}")
print(f"Avg. RAG Score: {df['rag_score'].mean()}")

Avg. LLM Only Score: 6.956
Avg. RAG Score: 7.266


In [12]:
print("Avg. Scores by Category:")
X = df.groupby(['category'])[['llm_only_score', 'rag_score']].mean().reset_index()
X['config'] = "test_results_llama_3_8B_instruct_bge-base-en-v1.5__768_1024_50_20_5"
X['overall_avg_llm_only_score'] = df['llm_only_score'].mean()
X['overall_avg_rag_score'] = df['rag_score'].mean()
X

Avg. Scores by Category:


,category,llm_only_score,rag_score,config,overall_avg_llm_only_score,overall_avg_rag_score
0,F-1,6.714286,7.571429,test_results_llama_3_8B_instruct_bge-base-en-v...,6.956,7.266
1,Green Card,7.875000,6.187500,test_results_llama_3_8B_instruct_bge-base-en-v...,6.956,7.266
2,H-1B,6.464286,7.464286,test_results_llama_3_8B_instruct_bge-base-en-v...,6.956,7.266
3,O Visa,6.475000,7.537500,test_results_llama_3_8B_instruct_bge-base-en-v...,6.956,7.266
4,RFE,8.083333,7.166667,test_results_llama_3_8B_instruct_bge-base-en-v...,6.956,7.266


In [13]:
X.to_csv("score_results_llama_3_8B_instruct_bge-base-en-v1.5__768_1024_50_20_5.csv", index=False)